In [ ]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsgluedq.transforms import EvaluateDataQuality

args = getResolvedOptions(sys.argv, ['JOB_NAME'])
sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
job.init(args['JOB_NAME'], args)

# Default ruleset used by all target nodes with data quality enabled
DEFAULT_DATA_QUALITY_RULESET = """
    Rules = [
        ColumnCount > 0
    ]
"""

# Script generated for node albums
albums_node1752882689693 = glueContext.create_dynamic_frame.from_options(format_options={"quoteChar": "\"", "withHeader": True, "separator": ","}, connection_type="s3", format="csv", connection_options={"paths": ["s3://myprojectonglue/staging/albums.csv"], "recurse": True}, transformation_ctx="albums_node1752882689693")

# Script generated for node artist
artist_node1752882732016 = glueContext.create_dynamic_frame.from_options(format_options={"quoteChar": "\"", "withHeader": True, "separator": ","}, connection_type="s3", format="csv", connection_options={"paths": ["s3://myprojectonglue/staging/artists.csv"], "recurse": True}, transformation_ctx="artist_node1752882732016")

# Script generated for node tracks
tracks_node1752882778505 = glueContext.create_dynamic_frame.from_options(format_options={"quoteChar": "\"", "withHeader": True, "separator": ","}, connection_type="s3", format="csv", connection_options={"paths": ["s3://myprojectonglue/staging/track.csv"], "recurse": True}, transformation_ctx="tracks_node1752882778505")

# Script generated for node Join album and artist
Joinalbumandartist_node1752883432570 = Join.apply(frame1=albums_node1752882689693, frame2=artist_node1752882732016, keys1=["artist_id"], keys2=["id"], transformation_ctx="Joinalbumandartist_node1752883432570")

# Script generated for node Join with track
Joinwithtrack_node1752883677789 = Join.apply(frame1=Joinalbumandartist_node1752883432570, frame2=tracks_node1752882778505, keys1=["track_id"], keys2=["track_id"], transformation_ctx="Joinwithtrack_node1752883677789")

# Script generated for node Drop Fields
DropFields_node1752883920936 = DropFields.apply(frame=Joinwithtrack_node1752883677789, paths=["track_id", "id"], transformation_ctx="DropFields_node1752883920936")

# Script generated for node datawarehouse
EvaluateDataQuality().process_rows(frame=DropFields_node1752883920936, ruleset=DEFAULT_DATA_QUALITY_RULESET, publishing_options={"dataQualityEvaluationContext": "EvaluateDataQuality_node1752882553778", "enableDataQualityResultsPublishing": True}, additional_options={"dataQualityResultsPublishing.strategy": "BEST_EFFORT", "observations.scope": "ALL"})
datawarehouse_node1752884229906 = glueContext.write_dynamic_frame.from_options(frame=DropFields_node1752883920936, connection_type="s3", format="glueparquet", connection_options={"path": "s3://myprojectonglue/datawarehouse/", "partitionKeys": []}, format_options={"compression": "snappy"}, transformation_ctx="datawarehouse_node1752884229906")

job.commit()